## Gmarket 상품

In [30]:
from bs4 import BeautifulSoup

with open('./gmarket/1/gmarket_1.html', 'r', encoding='utf-8') as file:
    html_content = file.read()


soup = BeautifulSoup(html_content, 'html.parser')
options = soup.find_all('span', class_='item_tit')

li = []
for title in options:
    li.append(title.text.strip())

li = set(li) #delete distinct
li = list(li)
for i in range(len(li)):
    print(f"{i+1}  {li[i]}")

1  럭비 라운드 반팔/여름/반팔티/냉감티/쿨론/기능성/남성/여름티/여성/남자/티셔츠/냉감/셔츠/쿨티/무지
2  30수 베이직 카라티/남성/여름/반팔/티셔츠/반팔티/냉감티/라운드/쿨론/기능성/여름티/기능성티/여성/남자
3  쿨론 배색 메쉬/반팔티셔츠/남성/냉감티/여름/반팔티/라운드/기능성/반팔/여름티/기능성티/여성/남자
4  쿨론 V넥 반팔(4XL 5XL) 냉감 티셔츠 특대사이즈/반팔/무지티/여름/반팔티/냉감티/라운드/기능성/남성
5  30수 백신 라운드 티/남자/여름/반팔티/여성/면반팔티/냉감티/쿨론/기능성/반팔/남성/여름티/기능성티
6  남자플리츠반팔티/여성/면반팔티/무지/반팔/쿨링티/스포츠/반팔티/여름/냉감티/라운드/쿨론/기능성/남성
7  3D입체 공법 무지 반팔티/여름/냉감티/라운드/여성/남자/티셔츠/반팔/남성/기능성/냉감/셔츠/쿨티
8  브이넥차선 반팔티/냉감티/라운드/쿨론/여성/남자/티셔츠/반팔/남성/기능성/냉감/셔츠/쿨티/무지
9  남여 공용국산 면 반팔티셔츠 단체 중고등학생티/반팔티/여성/남자/티셔츠/반팔/남성/기능성/냉감/셔츠
10  쿨론라운드 4XL 5XL 냉감 쿨론티 특대사이즈 빅사이즈/여성/면반팔티/무지/반팔/쿨링티/여름/반팔티/냉감티
11  시원한 여름티 브이넥 반팔티 아이스쿨티 냉장고티/여름/냉감티/라운드/쿨론/기능성/반팔/남성/기능성티
12  30수 T C PK 폴로/스포츠/반팔티/남성/여름/반팔/냉감티/라운드/쿨론/기능성/여름티/기능성티/여성/남자
13  퍼니-스윙 루즈핏 반팔 티셔츠/남자/여름/반팔티/여성/면반팔티/무지/냉감티/라운드/쿨론/기능성/남성
14  멘즈무지반팔티/라운드/반팔티/쿨론/기능성/반팔/여성/남자/티셔츠/남성/냉감/셔츠/쿨티/무지
15  100면오버사이즈무지티 면티 반팔티 이너티 레이어드/여름/냉감티/여성/남자/티셔츠/반팔/남성/기능성/냉감
16  (폴밋)데일리 기본 디자인 남성 반팔 면 티셔츠/여름/반팔티/여성/남자/기능성/냉감/셔츠/쿨티/무지
17  반팔 티셔츠 의류 T셔츠/여름/반팔티/냉감티/라운드/쿨론/기

## gmarket 옵션

In [31]:
from bs4 import BeautifulSoup

# HTML 파일 로드
with open('./gmarket/1/gmarket_1.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# BeautifulSoup 객체 생성
soup = BeautifulSoup(html_content, 'html.parser')

# <li class=""> 내의 <a> 태그 찾기
list_items = soup.find_all('li', class_='')  # 빈 class 속성을 가진 li 태그 모두 찾기

# 각 <li> 내의 <a> 태그에서 data-optionvalue1 속성 추출
for item in list_items:
    link = item.find('a', attrs={'data-optionvalue1': True})
    if link:  # link가 존재하면
        print(link['data-optionvalue1'])

M
XL
M
XL


# Gmarket (복수 옵션)

In [7]:
from bs4 import BeautifulSoup

with open('gmarket_example3.html', 'r', encoding='utf-8') as file:
    html_content = file.read()


soup = BeautifulSoup(html_content, 'html.parser')
options = soup.find_all('span', class_='text__name')
li = []
for title in options:
    li.append(title.text.strip())
li = set(li) #delete distinct

for option in li:
    print(option)

17.소프트린넨반팔티셔츠7종세트
06.여성면네추럴티셔츠7종세트
33.카라슬림핏티블라우스4종세트
46.텐셀5부데님반바지3종세트
29.레이스시스루자켓
30.후드데일리셔링점퍼
42.여성소프트면반팔티셔츠7종세트
01.여성베이직린넨티셔츠7종세트
19.오가닉면데일리니트4종세트
43.썸머모달티블라우스5종세트
41.데일리레터링티셔츠5종세트
38.프렌치린넨100반바지1종
20.세미시스루니트집업3종세트
31.여성라이크라코튼팬츠3종세트
27.여성데일리반팔티셔츠7종세트
36.아세테이트스트링쿨팬츠2종세트
34.썸머플리츠밴딩팬츠2종세트
02.썸머RN밴딩팬츠4종세트
22.남성맨투맨반팔티셔츠4종세트
18.시원한인견셔링블라우스3종세트
03.카라넥케이블반팔니트3종세트
11.시원한니트웨어3종세트
44.여성쿨레터링티셔츠5종세트
05.소프트실크텐셀반팔니트3종세트
07.시원한인견니트3종+나시1종
45.썸머와이드팬츠3종세트
23.여름크링클셋업4종(자켓+나시+팬츠+스커트)
25.코튼라이크라팬츠3종세트
15.프렌치린넨포켓자켓
04.프렌치린넨반바지3종세트
16.플레어데님팬츠+체인벨트
49.썸머인견와이드팬츠3종세트
26.썸머핀턱와이드핏팬츠3종세트
28.프렌치린넨100블라우스2종세트
50.여성PCM나일론팬츠3종세트
08.여성레이온카라니트3종세트
10.스트레이트핏린넨팬츠3종세트
40.스트레치시원한팬츠2종세트
24.여성기능성트레이닝셋업4종세트
21.남성레이온카라니트3종세트
48.린넨브이넥카라반팔니트4종세트
39.프렌치린넨세미와이드핏팬츠1종
13.프렌치린넨블렌드반팔티셔츠7종세트
35.클래식아웃핏셔츠3종세트
37.여성코튼데일리티셔츠7종세트
47.프렌치린넨카라넥니트3종세트
32.여성썸머아웃도어팬츠3종세트
09.린넨A라인버뮤다팬츠3종세트
14.반팔니트3종+이너탑(총4종)
12.시원한쿨링밴딩팬츠3종세트


# Coupang

In [1]:
from bs4 import BeautifulSoup

with open('coupang_example.html', 'r', encoding='utf-8') as file:
    html_content = file.read()


soup = BeautifulSoup(html_content, 'html.parser')